In [3]:
import pandas as pd
import json 

with open('datasets/genre.json') as f:
    genres = json.load(f)

with open('datasets/genres_one.json') as f:
    genres_one = json.load(f)
    
csvs = ['lyrics2.csv', 'lyrics3.csv']
df = pd.read_csv('datasets/lyrics1.csv')

for csv in csvs:
    df = df.append(pd.read_csv('datasets/' + csv))

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/genre.json'

In [2]:
drop = set(genres.keys()).difference(set(genres_one.keys()))

for k in drop:
    df = df[df.artist != k]

y = []
X = []
for _ , row in df.iterrows():
    X.append(row['lyrics'])
    y.append(genres_one[row['artist']])

NameError: name 'genres' is not defined

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from collections import defaultdict

class LyricsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tag_map = defaultdict(lambda : wordnet.NOUN)
        self.tag_map['J'] = wordnet.ADJ
        self.tag_map['V'] = wordnet.VERB
        self.tag_map['R'] = wordnet.ADV
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        new_X = [document.lower() for document in X]
        new_X = [word_tokenize(document) for document in new_X]
        
        for i, document in enumerate(new_X):
            words_transformed = []
            word_lemmatized = WordNetLemmatizer()
            for w, tag in pos_tag(document):
                if w not in stopwords.words('english') and w.isalpha():
                    word = word_lemmatized.lemmatize(w,self.tag_map[tag[0]])
                    words_transformed.append(word)
            new_X[i] = ' '.join(words_transformed)
        return new_X

In [7]:
from sklearn.pipeline import Pipeline

preperation = Pipeline([
    ('lyrics', LyricsTransformer())
])

X_prepared = preperation.fit_transform(X)

KeyboardInterrupt: 

In [40]:
# template for transforming text 

s = []
temp = ''
filename = 'Still_D.R.E.txt'
with open(filename) as file:
    while (line := file.readline()):
        temp += ' ' + line
s.append(temp)

s = preperation.fit_transform(s)

with open(filename, 'w') as f:
     for item in s:
        f.write("%s\n" % item)

In [6]:
with open('datasets/X_data.txt', 'r') as f:
    for item in X_prepared:
        f.write("%s\n" % item)
        
with open('datasets/y_data.txt', 'r') as f:
    for item in y:
        f.write("%s\n" % item

In [41]:
X = []
y = []

with open('datasets/X_data.txt') as file:
    while (line := file.readline()):
        X.append(line)
        
with open('datasets/y_data.txt') as file:
    while (line := file.readline()):
        y.append(line)

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(X)

X_train_tfidf = Tfidf_vect.transform(X_train)
X_test_tfidf = Tfidf_vect.transform(X_test)

In [44]:
from sklearn import svm, naive_bayes

nb = naive_bayes.MultinomialNB()
nb.fit(X_train_tfidf,y_train)# predict the labels on validation dataset

MultinomialNB()

In [47]:
import pickle
filename = 'models/nb.sav'
# pickle.dump(nb, open(filename, 'wb'))

In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear')
SVM.fit(X_train_tfidf,y_train) # predict the labels on validation datasetim

In [ ]:
import pickle
filename = 'models/SVM.sav'
# pickle.dump(SVM, open(filename, 'wb'))